In [1]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] == 300

False

# Data Exploration

In [2]:
# reading the dataset

data = pd.read_csv(r'C:\Github\Real-Estate-Price-Prediction\Real-Estate-Price-Prediction\Data\house_prices.csv')

In [3]:
# looking at the data

display(data.head(5),data.sample(5),data.tail(5))

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
7889,Super built-up Area,Ready To Move,B Narayanapura,2 BHK,NaN,1020,2.0,1.0,49.0
8439,Built-up Area,17-Dec,Sarjapur Road,2 BHK,NaN,1060,2.0,1.0,50.0
11012,Built-up Area,Ready To Move,Kengeri Satellite Town,2 BHK,MMowsik,1110,2.0,1.0,43.0
6959,Built-up Area,Ready To Move,Herohalli,3 Bedroom,NaN,1100,3.0,1.0,110.0
2325,Built-up Area,18-Mar,Sahakara Nagar,3 BHK,NaN,1655,3.0,1.0,115.0


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.0
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.0
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.0
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.0
13319,Super built-up Area,Ready To Move,Doddathoguru,1 BHK,NaN,550,1.0,1.0,17.0


In [4]:
# checking the data summary
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


**Inference:**

- there are 13320 records with 8 columns.
- total_sqft, bath, balcony, price are the float64 values rest are of object data type.
- there seem to have misssing values in the data.

In [5]:
# checking the central tencendy of the data

data.describe().T

,count,mean,std,min,25%,50%,75%,max
bath,13247.0,2.692610,1.341458,1.0,2.0,2.0,3.0,40.0
balcony,12711.0,1.584376,0.817263,0.0,1.0,2.0,2.0,3.0
price,13320.0,112.565627,148.971674,8.0,50.0,72.0,120.0,3600.0


**Inference:**

- There is a posibility of outliers as there is property with 40 number of baths which is extremely rare. Every property has minimum 1 bathrom with average of 2 bathrooms.
- There number of balcony varies from minimum 0 to maximum 3.
- The price column is not interpretable at the moment.

# Data Cleaning:

In [6]:
# checking for missing values
data.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

- We will be dropping availiblity, area_type, and society columns as they will not help to predict prices and overall predictive ability.

- we will be droping society column as well, as it have too many missing values.

- we will be treating the other columns as required.

In [7]:
df = data.drop(['availability','area_type','society','balcony'],axis = 1)

We will be droping the other columns missing values as they are small in number.

In [8]:
# droping the na values
df = df.dropna()
# checking the data
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [9]:
df.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

We can see some inconsistency in size column. We will be checking the same for other columns as well.

In [10]:
display(df['size'].unique(),df['total_sqft'].unique().sum(),
        df['bath'].unique()
        ,df['price'].unique())

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

'105626001440152112001170273233001310102018002785100011002250117511801540277060017552800176751012506601610115110252100 - 285010751760169319257001070172412901143129612541330.74970145980086912701670201011851600150014078404395845570011603000114012201350100550013581569124020891206115025114604400166013261325149916657081060710145028941330250265024001007966163016407821260141311161530370024971436276142720613067 - 81562650128210509459501870880153513601042 - 11051280500030501563.051167400018288901612103417109572795112517352050375010631904420020001145 - 1340142514701300450115215504007057701242170021441704184613401015 - 154013271186178314009801285912122519091359120717362850159517981475158012953600589141517879842405108019008051153114811101933350064516449101577405024209001108304529001162103514641866180491318688831664202612104111176212528611420149010841015101710271069134914171863101018475251850143815608501113138511282390164511922135117331221230111508159213886303252130853012059301380248311662023.71193

array([ 2.,  5.,  3.,  4.,  6.,  1.,  9.,  8.,  7., 11., 10., 14., 27.,
       12., 16., 40., 15., 13., 18.])

array([ 39.07, 120.  ,  62.  , ...,  40.14, 231.  , 488.  ])

In [11]:
# creating new column with just the number of bedrooms and adding it in the dataset
df['bhk'] = df['size'].apply(lambda x:int(x.split(' ')[0]))
# droping the orignal size column 
df = df.drop(['size'],axis = 1)
df.head()

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2


In the total_sqft column there are values containing the ranges of so we will be avaeraging the range and work with the mean value.

In [12]:
def is_float(x):
    try:
        float(x)
        return True
    except:
        return False

In [13]:
df[~df['total_sqft'].apply(is_float)].sample(5)

,location,total_sqft,bath,price,bhk
4226,Whitefield,524 - 894,1.0,34.385,1
6229,Bannerghatta Road,1365 - 1700,3.0,84.290,3
1183,Magadi Road,884 - 1116,2.0,46.500,2
8312,Tumkur Road,596 - 861,1.0,31.315,1
3890,Tumkur Road,1210 - 1477,3.0,57.760,3


In [14]:
# creating a function tp avergae the ranges 

def convert_sqft(x):
    token = x.split('-')
    if len(token) == 2:
        return (float(token[0])+float(token[1]))/2
    try:
        return float(x)
    except:
        return None

In [15]:
# applying the function and storing it 
df['total_sqft'] = df['total_sqft'].apply(convert_sqft)
df.sample(5)

,location,total_sqft,bath,price,bhk
11429,Thanisandra,1226.0,2.0,65.0,2
9029,Basaveshwara Nagar,900.0,2.0,60.0,2
1442,Ashwathnagar,2197.0,4.0,280.0,3
11325,"Thanisandra Main Road,",1308.0,2.0,86.0,2
10227,Rajaji Nagar,1222.0,2.0,98.0,2


In [16]:
# dropping these values as well
display(df.shape)
df = df.dropna()
df.shape

(13246, 5)

(13200, 5)

In [17]:

df.bhk.unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [18]:
# looking at all the records that have more than 20 bedrooms
df[df['bhk']>20]

,location,total_sqft,bath,price,bhk
1718,2Electronic City Phase II,8000.0,27.0,230.0,27
4684,Munnekollal,2400.0,40.0,660.0,43


If we look above we the the area of 43 and 27 bhk are way to less to acomodate 43 rooms with 40 bathrooms and 27 rooms with 27 bathrooms. so we will romove such descripancies as well.  

In [19]:

print(df.shape)
# removing all the column that have discrepencies in total_sqft
df = df[~(df['total_sqft']/df['bhk']<450)]
df.shape


(13200, 5)


(10627, 5)

In [20]:
# creating the price per sqft column
df['price_per_sqft'] = df['price']*100000/df['total_sqft']
df.sample(5)

,location,total_sqft,bath,price,bhk,price_per_sqft
3672,Whitefield,1269.0,2.0,75.0,2,5910.165485
6687,Whitefield,1435.0,3.0,45.0,3,3135.888502
8798,Kothanur,1786.0,2.0,106.0,3,5935.050392
4857,Whitefield,1778.0,3.0,122.0,3,6861.642295
13048,Sarjapur Road,1984.0,4.0,148.0,3,7459.677419


In [21]:

#We can certainly perform some dimensionality reduction on location column because as for a categorical column, 
# it has a lot of distinct values


df.location.nunique()

1088

In [23]:

# So we can classify those locations as other locations
location_grouped = df.groupby('location')['location'].agg('count').sort_values(ascending=False)
other_locations = location_grouped[location_grouped<=10]

In [24]:

display(df.location.nunique())
df.location = df.location.apply(lambda x: 'other' if x in other_locations else x)
df.location.nunique()

1088

194


Now we have 194 column, which is good because when converting it into categorical data, we won't have to deal with 1000 new columns, 200 is much more managable